In [2]:
import akshare as ak
import numpy as np

In [3]:
#日线换为周线数据
def transferToWeekLine(df, period='W'):
    data1 = df
    stock_data = pd.DataFrame(data1)

    #设定转换周期period_type  转换为周是'W',月'M',季度线'Q',五分钟'5min',12天'12D'
    stock_data["date"] = pd.to_datetime(stock_data["date"])
    period_type = period

    stock_data.set_index('date', inplace=True)

    #进行转换，周线的每个变量都等于那一周中最后一个交易日的变量值

    period_stock_data = stock_data.resample(period_type).last()

    #周线的volume和money等于那一周中volume和money各自的和

    period_stock_data['chg_pct'] = stock_data['chg_pct'].resample(period_type).last()

    #股票在有些周一天都没有交易，将这些周去除

    period_stock_data = period_stock_data[period_stock_data['chg_pct'].notnull()]

    period_stock_data.reset_index(inplace=True)

    data = np.array(period_stock_data)  #先将数据框转换为数组
    data_list = data.tolist()  #其次转换为列表
    for i in data_list:
        i[0] = str(i[0]).split(" ")[0]
    return data_list

In [4]:
#etf基本数据
fund_etf_fund_daily_em_df = ak.fund_etf_fund_daily_em()
print(fund_etf_fund_daily_em_df)

       基金代码                  基金简称      类型 2023-06-12-单位净值 2023-06-12-累计净值   
0    511010        国泰上证5年期国债ETF行情  债券型-长债        131.5510          1.3280  \
1    511260       国泰上证10年期国债ETF行情  债券型-长债        122.1140          1.2210   
2    159972         鹏华中证5年地债ETF行情  债券型-长债        114.9039          1.1512   
3    511270        海富通上证10年期ETF行情  债券型-长债        110.0481          1.2387   
4    511020   平安中证5-10年国债活跃券ETF行情  债券型-长债        109.2914          1.1459   
..      ...                   ...     ...             ...             ...   
766  513390  博时纳斯达克100ETF(QDII)行情    QDII             ---             ---   
767  159659  招商纳斯达克100ETF(QDII)行情    QDII             ---             ---   
768  159660       汇添富纳斯达克100ETF行情    QDII             ---             ---   
769  159507           广发国证通信ETF行情  指数型-股票             ---             ---   
770  159501  嘉实纳斯达克100ETF(QDII)行情    QDII             ---             ---   

    2023-06-09-单位净值 2023-06-09-累计净值     增长值    增长率        市价    折价率  
0    

In [10]:
import pandas as pd
import backtrader as bt

from datetime import datetime

import akshare as ak
% matplotlib inline

class Boll_strategy(bt.Strategy):
    #自定义参数，每次买入1800手
    params = (('size', 1800),)

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        ##使用自带的indicators中自带的函数计算出支撑线和压力线，period设置周期，默认是20
        self.lines.top = bt.indicators.BollingerBands(self.datas[0], period=20).top
        self.lines.bot = bt.indicators.BollingerBands(self.datas[0], period=20).bot

    def next(self):
        if not self.position:
            if self.dataclose <= self.lines.bot[0]:
                #执行买入
                self.order = self.buy(size=self.params.size)
            else:
                if self.dataclose >= self.lines.top[0]:
                    #执行卖出
                    self.order = self.close(size=self.params.size)


# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="600309", adjust="hfq").iloc[:, :6]
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    'date',
    'open',
    'close',
    'high',
    'low',
    'volume',
]
# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])
start_date = datetime(1991, 4, 3)  # 回测开始时间
end_date = datetime(2022, 6, 16)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df,
                           fromdate=start_date,
                           todate=end_date)  # 加载数据
# 初始化cerebro回测系统设置
cerebro = bt.Cerebro()
#将数据传入回测系统
cerebro.adddata(data)
# 将交易策略加载到回测系统中
cerebro.addstrategy(Boll_strategy)
# 设置初始资本为10,000
startcash = 1000000
cerebro.broker.setcash(startcash)
# 设置交易手续费为 0.25%
cerebro.broker.setcommission(commission=0.001)
#运行回测系统
from core.backtrade.analysis.console import ConsoleAnalyzer
from config.config import TradeConfig

config = TradeConfig()
ConsoleAnalyzer.execute(cerebro, config)




无交易记录
分析报告
股票： ['']
初始资金： 1000000
结束资产： 5519760.22
累计收益： 4.519760219999996
年化收益： 0.08987785307643303
最大回撤： -0.4684236300423941
夏普比： 0.5524816331193187
卡玛比： 0.1918730126153088
omega： 1.112758738946507
交易次数： 0
交易胜率： 0
最大连赢次数： 0
最大连输次数： 0
